In [ ]:
!pip install datasets

from datasets import load_dataset
import requests
import json
import pandas as pd
from datasets import Dataset

import numpy as np
import torch
from torch.utils.data import DataLoader
import logging
from tqdm import tqdm
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import RandomSampler, SequentialSampler

import time
import datetime
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import random
import re
import copy

In [ ]:
# hyperparams
batch_size = 2
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 100
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# torch Dataset
class GPT2Dataset(torch.utils.data.Dataset):

  def __init__(self, txt_list, tokenizer, max_length=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>' + txt['prompt'] + txt['answer'], truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

# helper methods
def train_valid_test(task_folder='multiArith/'):
  '''split easy-cot data with 8:1:1 ratio'''
  raw = load_dataset("json", data_files="easy_cot/" + task_folder + "data.json")
  data = Dataset.from_pandas(pd.DataFrame(data=raw))
  train_test_split = data.train_test_split(test_size=0.2, shuffle=False, seed=42)
  train = train_test_split['train']
  test = train_test_split['test']
  valid_test_split = test.train_test_split(test_size=0.5, shuffle=False, seed=42)
  valid = valid_test_split['train']
  test = valid_test_split['test']
  return train['train'], valid['train'], test['train']


def save_model(model, tokenizer, output_dir):
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  print("Saving model to %s" % output_dir)

  model_to_save = model.module if hasattr(model, 'module') else model
  model_to_save.save_pretrained(output_dir)
  tokenizer.save_pretrained(output_dir)


def load_model(output_dir):
  model = GPT2LMHeadModel.from_pretrained(output_dir)
  tokenizer = GPT2Tokenizer.from_pretrained(output_dir)
  model.to(device)
  return model, tokenizer

def extract_answer(output, is_generated=False):
  if is_generated:
    pattern = r'-->(.*)'
  else:
    pattern = r'-->(.*?)<'
  m = re.search(pattern, output)
  if m:
      return m.group(1)
  else:
      return None

def get_accuracy(model, tokenizer, test_set):
  model.eval()
  correct = 0
  total = 0

  for sample in test_set:
    prompt = sample['prompt']
    answer = extract_answer(sample['answer'])
    len_prompt = len(prompt)

    generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    generated = generated.to(device)

    sample_outputs = model.generate(
                                    generated,
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    max_new_tokens=50,
                                    num_return_sequences=1,
                                    pad_token_id=tokenizer.eos_token_id
                                    )

    for i, sample_output in enumerate(sample_outputs):

      # print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
      # print(tokenizer.decode(sample_output,skip_special_tokens=True)[len_prompt:])
      extracted_answer = extract_answer(tokenizer.decode(sample_output, skip_special_tokens=True)[len_prompt:], is_generated=True)

      # print(f'extracted: {extracted_answer}, truth: {answer}')
      if extracted_answer is not None and extracted_answer.strip() == answer.strip():
        correct += 1

      total += 1

  return correct/total


def three_run(model, tokenizer, test_set):
  avg_accuracy = 0
  for _ in range(3):
    avg_accuracy += get_accuracy(model, tokenizer, test_set)
  avg_accuracy /= 3
  print(f'Average accuracy over 3 runs: {avg_accuracy}')

def run_test(model, tokenizer, valid, test):
  # print("")
  # print("====== Experiment Results =======")
  # print("Running Default GPT2 on Task ...")
  # default_model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration).to(device)
  # default_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
  # print('validation--> ', end='')
  # three_run(default_model, default_tokenizer, valid)
  # print('test--> ', end='')
  # three_run(default_model, default_tokenizer, test)
  model.eval()
  print("")
  print("======== Start of Results ========")
  if valid is not None:
    print('validation--> ', end='')
    three_run(model, tokenizer, valid)
  print('test--> ', end='')
  three_run(model, tokenizer, test)
  print("======== End of Results ========")

## Experiment

In [ ]:
def test_all(task_folder='multiArith/', model_dir='/kaggle/input/easycot_coinflip/other/hf/1/coinFlip'):
    ''' Run Auto-CoT, Fine-tune-CoT, Easy-CoT on same task'''
    default_configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
    default_model = GPT2LMHeadModel.from_pretrained("gpt2", config=default_configuration).to(device)
    default_tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
    model, tokenizer = load_model(model_dir)

    raw = load_dataset("json", data_files="/kaggle/input/easy-cot/easy_cot/" + task_folder + "data.json", field="data")
    # test = context-augmented, test_cpy = original prompt
    data = Dataset.from_pandas(pd.DataFrame(data=raw))
    train_test_split = data.train_test_split(test_size=0.2, shuffle=False, seed=42)
    test = train_test_split['test']['train']
    test_cpy = copy.deepcopy(test)
    # load demos
    raw = load_dataset("json", data_files="/kaggle/input/easy-cot/easy_cot/" + task_folder + "demos.json", field="data")
    demos = raw['train']

    context = ""
    for sample in demos:
        tmp_str = context + sample['demo'] + "\n"
        if len(tmp_str.split()) > 600:
            break
        context = tmp_str

    for sample in test:
        sample['prompt'] = context + sample['prompt']

    print("")
    print("Running Fine-tune-CoT...")
    run_test(model, tokenizer, None, test_cpy)
    print("")
    print("Running Auto-CoT...")
    run_test(default_model, default_tokenizer, None, test)
    print("")
    print("Running Easy-CoT...")
    run_test(model, tokenizer, None, test)
    print("")


# Results

In [ ]:
test_all('coinFlip/','/kaggle/input/easycot_coinflip/other/hf/1/coinFlip')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.020202020202020204
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.04040404040404041
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.5151515151515151
======== End of Results ========



In [ ]:
test_all('lastLetter/','/kaggle/input/easycot_letter/other/hf/1/lastLetter')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0
======== End of Results ========



In [ ]:
test_all('commonSenseQA/','/kaggle/input/easycot_csqa/other/hf/1/commonSenseQA')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.04655408489274304
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.018256503879507075
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.18575992697398447
======== End of Results ========



In [ ]:
test_all('strategyQA/','/kaggle/input/easycot_sqa/other/hf/1/strategyQA')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.06345733041575492
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.024070021881838075
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0
======== End of Results ========



In [ ]:
test_all('multiArith/','/kaggle/input/easycot_ma/other/hf/1/multiArith')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.008403361344537815
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.01680672268907563
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.025210084033613446
======== End of Results ========



In [ ]:
test_all('gsm8k/','/kaggle/input/easycot_gsm8k/other/hf/1/gsm8k')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


Running Fine-tune-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0028457598178713715
======== End of Results ========

Running Auto-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.0011383039271485487
======== End of Results ========

Running Easy-CoT...

======== Start of Results ========
test--> Average accuracy over 3 runs: 0.008537279453614115
======== End of Results ========

